In [1]:
import numpy as np, pandas as pd, xarray as xr
from pathlib import Path
import datetime, networkx as nx, yaml, warnings, sys, logging, shutil
from disjoint_set import DisjointSet
from helper import singleglob, nxrender, Step, dict_merge, PdfWriter, DictWriter, TableWriter, TextWriter
import matplotlib.pyplot as plt
import matplotlib as mpl, seaborn as sns

In [2]:

session_folder = _ #"/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_MT/Rat101_20240801/"
dest_folder = _ #"/home/julienb/Documents/Data/Luisa/"
force = _ #False

In [3]:
# Parameters
dest_folder = "/home/julienb/Documents/Data/Luisa/Rat241/opto_MT/Session--Rat241_20240730"
session_folder = "/media/filer2/T4/Clara/Data/Luisa/Rat241/opto_MT/Rat241_20240730"


In [4]:
session_folder = Path(session_folder)
dest_folder =  Path(dest_folder) 
templates_folder = Path("..").resolve()
session_folder, dest_folder, templates_folder


(PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat241/opto_MT/Rat241_20240730'),
 PosixPath('/home/julienb/Documents/Data/Luisa/Rat241/opto_MT/Session--Rat241_20240730'),
 PosixPath('/home/julienb/Documents/Luisa/scripts_and_templates'))

In [5]:
poly_folder = dest_folder / "poly"
fiber_folder = dest_folder / "fiber"
spike2_folder = dest_folder / "spike2"
video_folder = dest_folder / "video"
poly_folder.mkdir(exist_ok=True, parents=True)
fiber_folder.mkdir(exist_ok=True, parents=True)
spike2_folder.mkdir(exist_ok=True, parents=True)
video_folder.mkdir(exist_ok=True, parents=True)


In [6]:
metadata_path = session_folder/"metadata.yaml"
metadata = yaml.safe_load(metadata_path.open("r"))
metadata

{'experimenter': {'name': 'Luisa Abels'},
 'date': '2024/07/30',
 'project_name': 'Dopa-Move',
 'subject': {'name': 'Rat241',
  'cre-line': 'TH-Cre',
  'sex': 'F',
  'handedness': 'Right',
  'opsin': 'Jaws',
  'opsin_injection_date': '2024/05/28',
  'opsin_location': 'SNc'},
 'recordings': {'fiber': {'channels': {'CH-1': {'brain_location': 'Left-DLS'},
    'CH-2': {'brain_location': 'Right-DLS'}},
   'tool': None},
  'electrophy': None,
  'behavior': 'Poly'},
 'task': {'name': 'HF_55_RandomTrial_NoGoGo_Right_both_sound_nico_LaserJAWS_S1_MT1_1sec_NoPadCheck_2000',
  'position': 'head-fixed',
  'behavior': {'category': 'lever-press',
   'cases': {'go': ['go', 'nogo'],
    'mouvement': ['straight'],
    'stimulation': [True]}},
  'opto': {'name': None,
   'laser_protocol': {'type': 'continuous',
    'pulse_info': {'n_pulse': 4, 'frequency_Hz': 15}},
   'laser_intensity_mW': 15,
   'timing': {'event': 'pad_lift',
    'start_rel_event_ms': 0,
    'duration_ms': 1000,
    'end_event': 'lever

In [7]:
del metadata["data_files"]
if "behavior" in metadata["task"]:
    del metadata["task"]["behavior"]
if "behavior" in metadata["recordings"]:
    del metadata["recordings"]["behavior"]
metadata["recordings"]["poly"]={}
del metadata["recordings"]["fiber"]["tool"]
metadata["recordings"]["poly"]["valid_recording_intervals"] = [dict(start=None, end=None)]
metadata["task"]["cue_location"]=dict(LowPitch= "Left", HighPitch="Left")
metadata


{'experimenter': {'name': 'Luisa Abels'},
 'date': '2024/07/30',
 'project_name': 'Dopa-Move',
 'subject': {'name': 'Rat241',
  'cre-line': 'TH-Cre',
  'sex': 'F',
  'handedness': 'Right',
  'opsin': 'Jaws',
  'opsin_injection_date': '2024/05/28',
  'opsin_location': 'SNc'},
 'recordings': {'fiber': {'channels': {'CH-1': {'brain_location': 'Left-DLS'},
    'CH-2': {'brain_location': 'Right-DLS'}}},
  'electrophy': None,
  'poly': {'valid_recording_intervals': [{'start': None, 'end': None}]}},
 'task': {'name': 'HF_55_RandomTrial_NoGoGo_Right_both_sound_nico_LaserJAWS_S1_MT1_1sec_NoPadCheck_2000',
  'position': 'head-fixed',
  'opto': {'name': None,
   'laser_protocol': {'type': 'continuous',
    'pulse_info': {'n_pulse': 4, 'frequency_Hz': 15}},
   'laser_intensity_mW': 15,
   'timing': {'event': 'pad_lift',
    'start_rel_event_ms': 0,
    'duration_ms': 1000,
    'end_event': 'leverpress'}},
  'cue_location': {'LowPitch': 'Left', 'HighPitch': 'Left'}}}

In [8]:
task  = metadata["task"]["name"]
try:
    task_path = singleglob(session_folder, f"*{task}*.xls")
except FileNotFoundError:
    task_path = singleglob(templates_folder / "PolyEventTemplates",f"*{task}*.xls")
task_poly_template = singleglob(templates_folder / "PolyEventTemplates",f"*{task}*.yaml")
poly_template = yaml.safe_load(task_poly_template.open("r"))
poly_template


{'task': {'name': 'HF_55_RandomTrial_NoGoGo_Right_both_sound_nico_LaserJAWS_S1_MT1_1sec_NoPadCheck_2000',
  'subject_position': 'head-fixed',
  'cue_location': {'LowPitch': 'Right', 'HighPitch': 'Right'},
  'events': {'poly': [{'detection': {'poly_line_num': 33},
     'description': {'event': 'cue',
      'trial_type': {'stimulation': 'None',
       'go': False,
       'cue': 'WhiteNoise'}}},
    {'detection': {'poly_line_num': 41},
     'description': {'event': 'cue',
      'trial_type': {'stimulation': '1000ms', 'go': True, 'cue': 'LowPitch'}}},
    {'detection': {'poly_line_num': 7},
     'description': {'event': 'cue',
      'trial_type': {'stimulation': 'None', 'go': True, 'cue': 'LowPitch'}}},
    {'detection': {'poly_line_num': 12},
     'description': {'event': 'cue',
      'trial_type': {'stimulation': 'None', 'go': True, 'cue': 'HighPitch'}}},
    {'detection': {'poly_name': 'MT'},
     'description': {'event': 'mouvement_start'}},
    {'detection': {'poly_line_num': [45, 11,

In [9]:
fiber_template = yaml.safe_load((templates_folder/"fiber_event_template.yaml").open("r"))
fiber_template

[{'detection': {'name': 'Input1', 'duration_min': 0.13, 'duration_max': 0.17},
  'description': {'event': 'cue',
   'trial_type': {'stimulation': 'None',
    'go': False,
    'cue_type': 'WhiteNoise'}}},
 {'detection': {'name': 'Input1', 'duration_min': 0.18, 'duration_max': 0.22},
  'description': {'event': 'cue',
   'trial_type': {'stimulation': 'bilateral',
    'go': True,
    'cue_type': 'LowPitch'}}},
 {'detection': {'name': 'Input1', 'duration_min': 0.02, 'duration_max': 0.06},
  'description': {'event': 'cue',
   'trial_type': {'stimulation': 'None', 'go': True, 'cue_type': 'LowPitch'}}},
 {'detection': {'name': 'Input1', 'duration_min': 0.08, 'duration_max': 0.12},
  'description': {'event': 'cue',
   'trial_type': {'stimulation': 'None',
    'go': True,
    'cue_type': 'HighPitch'}}},
 {'detection': {'name': 'Input2'},
  'description': {'event': 'mouvement_start'}},
 {'detection': {'name': 'Input3', 'duration_min': 0.13, 'duration_max': 0.17},
  'description': {'event': 'rewar

In [10]:
dat_file = singleglob(session_folder, "**/*.dat")

In [11]:
task_template = poly_template["task"]["events"]["poly"]
for d in task_template:
    if "trial_type" in d["description"]:
        d["description"]["trial_type"]["cue_type"] = d["description"]["trial_type"]["cue"]
        if str(d["description"]["trial_type"]["stimulation"]).endswith("ms") or str(d["description"]["trial_type"]["stimulation"]).endswith("Hz"):
            d["description"]["trial_type"]["stimulation"] = "bilateral"
        del d["description"]["trial_type"]["cue"]
task_template.append(yaml.safe_load(
"""
description:
  event: trial_end
detection:
  poly_line_num: 1
"""
))

metadata["task"]["events"] = dict(poly=task_template, fiber=fiber_template)
metadata["date"] = datetime.datetime.strptime(dat_file.open("r").read(10), "%d/%m/%Y").strftime('%Y/%m/%d')
metadata

{'experimenter': {'name': 'Luisa Abels'},
 'date': '2024/07/30',
 'project_name': 'Dopa-Move',
 'subject': {'name': 'Rat241',
  'cre-line': 'TH-Cre',
  'sex': 'F',
  'handedness': 'Right',
  'opsin': 'Jaws',
  'opsin_injection_date': '2024/05/28',
  'opsin_location': 'SNc'},
 'recordings': {'fiber': {'channels': {'CH-1': {'brain_location': 'Left-DLS'},
    'CH-2': {'brain_location': 'Right-DLS'}}},
  'electrophy': None,
  'poly': {'valid_recording_intervals': [{'start': None, 'end': None}]}},
 'task': {'name': 'HF_55_RandomTrial_NoGoGo_Right_both_sound_nico_LaserJAWS_S1_MT1_1sec_NoPadCheck_2000',
  'position': 'head-fixed',
  'opto': {'name': None,
   'laser_protocol': {'type': 'continuous',
    'pulse_info': {'n_pulse': 4, 'frequency_Hz': 15}},
   'laser_intensity_mW': 15,
   'timing': {'event': 'pad_lift',
    'start_rel_event_ms': 0,
    'duration_ms': 1000,
    'end_event': 'leverpress'}},
  'cue_location': {'LowPitch': 'Left', 'HighPitch': 'Left'},
  'events': {'poly': [{'detectio

In [12]:
yaml.safe_dump(metadata, (dest_folder / "metadata.yaml").open("w"), sort_keys=False)

In [13]:
if not (poly_folder/f"task -- {task_path.name}").exists():
    shutil.copy(task_path, poly_folder/f"task -- {task_path.name}")


In [14]:
dat_file = singleglob(session_folder, "**/*.dat")
if not (poly_folder/f"events -- {dat_file.name}").exists():
    shutil.copy(dat_file, poly_folder/f"events -- {dat_file.name}")

In [15]:
for s in ["Events.csv", "Fluorescence.csv", "Fluorescence-unaligned.csv"]:
    if not (fiber_folder/s).exists():
        shutil.copy(singleglob(session_folder, f"**/{s}"), fiber_folder/s)

In [16]:
for f in session_folder.glob("**/*.sea"):
    if not (poly_folder / f.name).exists():
        shutil.copy(f, poly_folder / f.name)
for f in session_folder.glob("**/*.cpt"):
    if not (poly_folder / f.name).exists():
        shutil.copy(f, poly_folder / f.name)

In [17]:
sr2x = singleglob(session_folder, "**/*.s2rx")
if not (spike2_folder/f"channel_metadata -- {sr2x.name}").exists():
    shutil.copy(sr2x, spike2_folder/f"channel_metadata -- {sr2x.name}")
smrx = singleglob(session_folder, "**/*.smrx")
if not (spike2_folder/f"channel_data -- {sr2x.name}").exists():
    shutil.copy(smrx, spike2_folder/f"channel_data -- {sr2x.name}")

In [18]:
analysis_folder = dest_folder / "analysis"
analysis_folder.mkdir(exist_ok=True, parents=True)
